<a href="https://colab.research.google.com/github/willameoliveira/ibot-factory/blob/master/ProjetoImersaoIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projeto IBot Factory, fábrica de chatbot inteligente**
Respondendo apenas três perguntas usando o chatbot, gere um assistente virtual para seu negócio. O projeto ainda é um protótipo.

Será útil para qualquer instituição (seja de ensino, de saúde, ongs, etc) que deseja ter uma assistente inteligente para seu negócio. Um grande objetivo é democratizar a criação de assistentes virtuais principalmente nas organizações públicas, melhorando a qualidade do atendimento.

Para executar esse notebook, observe que há vários trechos que só precisam ser executados apenas uma vez.

O código inicia fazendo as instalações necessárias (biblioteca da ia generativa do google, biblioteca para acessar o Google Drive a partir do python e, por último, uma biblioteca para trabalhar com PDF no python respectivamente). Ele só precisa ser executado apenas uma vez por notebook.

In [ ]:
!pip install -q -U google-generativeai
!pip install PyDrive
!pip install PyPDF2

Nesse trecho são feitos todos os imports de bibliotecas necessárias.

*   `google.generativeai as genai`: Importa a biblioteca de IA generativa do Google, que você pode usar para tarefas como geração de texto, sumarização e resposta a perguntas.
*   `from google.colab import userdata`: Importa o módulo userdata do Google Colab, que pode ser usado para acessar armazenamento persistente dentro do seu ambiente do Colab.
*   `from urllib.parse import urlparse`: Importa a função urlparse, útil para dividir URLs em seus componentes (esquema, netloc, caminho, etc.).
*  `import PyPDF2`: Importa a biblioteca PyPDF2, permitindo que você trabalhe com arquivos PDF, incluindo leitura, manipulação e gravação.
*   `from pydrive.auth import GoogleAuth` e `from pydrive.drive import GoogleDrive`: Importam componentes da biblioteca PyDrive, que fornece uma interface para interagir com sua conta do Google Drive a partir do Python. Você pode usá-la para enviar, baixar e gerenciar arquivos no Google Drive.
*   `from google.colab import auth` e `from oauth2client.client import GoogleCredentials`: Importam módulos necessários para autenticar seu notebook Colab com sua conta do Google, concedendo acesso aos seus arquivos do Google Drive e outros serviços do Google.

In [59]:
import google.generativeai as genai
from google.colab import userdata
from urllib.parse import urlparse
import PyPDF2
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

Aqui o cliente PyDrive é autenticado e criado, podendo acessar o Google Drive do usuário. Está sendo usada a credencial padrão, que pede acesso total aos arquivos do Drive. Ao executar esse trecho, será pedida a permissão para acesso total ao seu Drive. Não tenha medo de conceder esse acesso, observe que o código não remove nada do seu Drive, apenas lê o arquivo PDF que você mesmo informará. Como esse código é só um protótipo, essa credencial resolve por enquanto.
OBS: Esse trecho só precisa ser executado uma vez por notebook.

In [24]:
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Esse trecho corresponde ao acesso do dono do negócio e, por isso, só precisa ser executado uma vez por notebook. O público alvo do negócio que acessa apenas o assistente virtual, não terá acesso a essa parte. Por ser protótipo, só está sendo possível gerar uma única assistente virtual por vez.

In [ ]:
print("Seja bem vindo ao IBot Factory, seu criador de chatbot inteligente. 😁")
tipo_negocio = input("Primeiro informe como se identifica seu negócio (ex: empresa, organização ou instituição?): ")
razao_social = input("\nInforme o nome da sua organização da forma como deseja que apareça no assistente virtual: ")
link_arquivo = input("""

Agora informe o link do PDF contido no seu Google Drive que contém as informações dos seus
serviços (CUIDADO: Certifique-se de que todo o conteúdo possa ser publicado na internet e que não há conteúdo ofensivo).
Para isso, basta clicar com o botão direito no arquivo e escolher a opção copiar link para a área de transferência e
colar aqui: """)

# Analisa a URL
parsed_url = urlparse(link_arquivo)

# Extraindo o ID da url
file_id = parsed_url.query.split("=")[1].split("&")[0]

# criando arquivo vazio, apenas com o id do que se deseja acessar
downloaded = drive.CreateFile({'id': file_id})
# baixando o arquivo para o sistema de arquivo local
downloaded.GetContentFile("document0.pdf")

print("\nParabéns. As configurações iniciais do seu assistente já foram feitas!")

Inicializando o modelo Gemini. A partir daqui inicia a execução do Assistente Virtual que foi configurado pelo dono do negócio.

In [66]:
genai.configure(api_key=userdata.get('API_KEY'))

# Set up the model
generation_config = {
  "temperature": 0.5,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

O próximo trecho tem a função de extrair o texto de cada página do arquivo PDF recebido e retornar um dicionário de textos do PDF agrupado por página.

Em detalhes:
1. A função recebe o caminho do arquivo PDF (pathname) como entrada.
2. Cria uma lista `parts`: Essa lista armazenará o texto extraído de cada página, junto com marcadores para indicar o início do PDF e o começo de cada página.
3. Abre o arquivo PDF: A biblioteca PyPDF2 é usada para abrir o arquivo PDF especificado pelo pathname.
4. Itera pelas páginas do PDF: O código percorre cada página do PDF, extraindo seu conteúdo textual e adiciona na lista `pages`.
5. Extrai o texto e adiciona ao dicionário: O texto de cada página é extraído e adicionado ao dicionário `parts`. O dicionário também inclui marcadores como "--- START OF PDF {pathname} ---" para indicar o início do PDF e "--- PAGE {index+1} ---" para indicar o início de cada página.
6. Retorna o dicionário: A função retorna o dicionário `parts`, que contém o texto extraído de cada página do PDF, juntamente com os marcadores.



In [67]:
def extract_pdf_pages(pathname: str) -> list[str]:
  parts = [f"--- START OF PDF ${pathname} ---"]
  # Add logic to read the PDF and return a list of pages here.
  pages = []
  reader = PyPDF2.PdfReader(pathname)
  for p in reader.pages:
    pages.append(p.extract_text())

  for index, page in enumerate(pages):
    parts.append(f"--- PAGE {index} ---")
    parts.append(page)
  return parts

Inicializando o chat do Gemini. O conteúdo do PDF lido é adicionado como histórico do chat para que as próximas respostas sejam baseadas no conteúdo dele. Na versão final, ele será incluído usando Embeddings para melhor eficiência e economia de uso de tokens.

In [ ]:
chat = model.start_chat(history=[
  {
    "role": "user",
    "parts": extract_pdf_pages("/content/document0.pdf")#<path>/document0.pdf
  },
  {
    "role": "user",
    "parts": [f"""O arquivo contém informações
    sobre os serviços oferecidos pelo(a) {tipo_negocio} chamado(a) {razao_social}.
    Haja como uma atendente virtual dessa {tipo_negocio} e responda, em
    tom amigável e com uso de emojis, as perguntas que serão feitas apenas
    usando esse documento como base. Antes de começar a responder a primeira
    pergunta, se apresente."""]
  },
])

prompt = input(f"Digite algo (fim para parar): ")
while prompt != "fim":
  response = chat.send_message(prompt)
  print (f"{razao_social}: {response.text}")
  prompt = input(f"Digite algo: ")